# <center> Recommender Systems </center>
## <center>  Week 2. Basic baselines </center>

In [1]:
import pickle
import numpy as np
from math import sqrt
from tqdm import tqdm
import pandas as pd

## Словарь с предпочтениями {кинокритик: {фильм : оценка}}

Посмотрим на explicit данные - когда у нас есть оценки по пользователям. 

In [2]:
with open('critics_reviews', 'rb') as f:
    reviews = pickle.load(f)

reviews

{'Lisa Rose': {'Lady in the Water': 2.5,
  'Snakes on a Plane': 3.5,
  'Just My Luck': 3.0,
  'Superman Returns': 3.5,
  'You, Me and Dupree': 2.5,
  'The Night Listener': 3.0},
 'Gene Seymour': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 3.5,
  'Just My Luck': 1.5,
  'Superman Returns': 5.0,
  'The Night Listener': 3.0,
  'You, Me and Dupree': 3.5},
 'Michael Phillips': {'Lady in the Water': 2.5,
  'Snakes on a Plane': 3.0,
  'Superman Returns': 3.5,
  'The Night Listener': 4.0},
 'Claudia Puig': {'Snakes on a Plane': 3.5,
  'Just My Luck': 3.0,
  'The Night Listener': 4.5,
  'Superman Returns': 4.0,
  'You, Me and Dupree': 2.5},
 'Mick LaSalle': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 4.0,
  'Just My Luck': 2.0,
  'Superman Returns': 3.0,
  'The Night Listener': 3.0,
  'You, Me and Dupree': 2.0},
 'Jack Matthews': {'Lady in the Water': 3.0,
  'Snakes on a Plane': 4.0,
  'The Night Listener': 3.0,
  'Superman Returns': 5.0,
  'You, Me and Dupree': 3.5},
 'Toby': {'Snak

1. Сделаем первый бейзлайн **Top Popular**. Однако, популярность можно понять по-разному, например, как наибольшее число просмотров по всем пользователям. Дополнительно пофильтруем фильмы и выведем топ самых просматриваемых фильмов для пользователя, оставив только те, которые он еще не видел.

In [3]:
def get_pop_movies(reviews):
    movies_rating = {}
    for user, films in reviews.items():
        for film, _ in films.items():
            movies_rating.setdefault(film, 0)
            movies_rating[film] += 1
    movies_rating = dict(sorted(movies_rating.items(), key=lambda x: x[1], reverse=True))
    return movies_rating


def top_pop_recommender(reviews, person):
    recoms = get_pop_movies(reviews)
    recoms = {k: v for k, v in recoms.items() if k not in reviews[person]}
    return recoms

In [4]:
get_pop_movies(reviews)

{'Snakes on a Plane': 7,
 'Superman Returns': 7,
 'You, Me and Dupree': 6,
 'The Night Listener': 6,
 'Lady in the Water': 5,
 'Just My Luck': 4}

In [5]:
top_pop_recommender(reviews, 'Toby')

{'The Night Listener': 6, 'Lady in the Water': 5, 'Just My Luck': 4}

2. **Top Popular v.2.0**. Далеко не всегда наиболее часто просматриваемые - это самые лучшие по рейтингу фильмы. Можно посчитать топ фильмов по среднему рейтингу из отзывов пользователей. Сделаем рекомендации по топу популярности на основе средней оценки фильма:  

In [9]:
def get_movies_avg_scores(reviews):
    movies_rating = {}
    for user, films in reviews.items():
        for film, score in films.items():
            movies_rating.setdefault(film, [])
            movies_rating[film].append(score)

    movies_rating = {k: np.mean(v) for k, v in movies_rating.items()}
    movies_rating = dict(sorted(movies_rating.items(), key=lambda x: x[1], reverse=True))
    return movies_rating


def top_avg_scores_recommender(reviews, person):
    recoms = get_movies_avg_scores(reviews)
    new_movies = set(recoms.keys()).difference(reviews[person])
    recoms = {k: round(v, 2) for k, v in recoms.items() if k in new_movies}
    return recoms

In [10]:
get_movies_avg_scores(reviews)

{'Superman Returns': 4.0,
 'Snakes on a Plane': 3.7142857142857144,
 'The Night Listener': 3.4166666666666665,
 'Lady in the Water': 2.8,
 'You, Me and Dupree': 2.5,
 'Just My Luck': 2.375}

In [11]:
top_avg_scores_recommender(reviews, 'Toby')

{'The Night Listener': 3.42, 'Lady in the Water': 2.8, 'Just My Luck': 2.38}

+ Для некоторых доменов у нас будут допустимы повторы по объектам (например, товары в супермаркете). К тому же, Top popular - это неперсональные рекомендации. Как вы думаете, если повторы допустимы, какие можно сделать персонализированные бейзлайны на статистиках? При условии, что есть только интеракции (то есть, никакие дополнительные признаки не доступны). 

Дополнительно, простыми, но надеждными бейзлайнами будет посчитать рекомендации на основе сходства пользователей или объектов (user/item similarity). Подход обычно состоит из этапов:

* Нормализация оценок.
* Выбор и расчет метрики схожести.
* Выбор соседей для формирования множества рекомендаций.


3. **User-similarity**. Давайте рассмотрим такой вариант - нужно найти для $i$-ого пользователя $k$ похожих на него соседей (similarity user-based подход). По соседям соберем множество просмотренных фильмов и выведем топ $n$ рекомендаций в порядке убывания их среднего рейтинга по соседям, где оценка каждого соседа учитывается c весом его схожести с $i$-м пользователем. 

Вспомним самые простые и распространенные метрики схожести (расстояния): 

<img src=https://www.researchgate.net/profile/Barthelemy-Durette/publication/275954089/figure/tbl2/AS:614304592166937@1523473036482/Formulae-of-the-similarity-and-distance-measures.png>

In [45]:
def sim_distance(reviews, person1, person2):
    sum_of_squares = sum([(reviews[person1][item] - reviews[person2][item])**2
                         for item in reviews[person1] if item in reviews[person2]])
    return 1/(1 + np.sqrt(sum_of_squares))

Выведем попарные метрики схожести пользователей:

In [46]:
compared = set()
for first in reviews:
    for second in reviews:
        compared.add((first, second))
        if first != second and (second, first) not in compared:
            print(f"{first}'s & {second}'s similarity: ", round(sim_distance(reviews, first, second), 2))

Lisa Rose's & Gene Seymour's similarity:  0.29
Lisa Rose's & Michael Phillips's similarity:  0.47
Lisa Rose's & Claudia Puig's similarity:  0.39
Lisa Rose's & Mick LaSalle's similarity:  0.41
Lisa Rose's & Jack Matthews's similarity:  0.34
Lisa Rose's & Toby's similarity:  0.35
Gene Seymour's & Michael Phillips's similarity:  0.34
Gene Seymour's & Claudia Puig's similarity:  0.28
Gene Seymour's & Mick LaSalle's similarity:  0.28
Gene Seymour's & Jack Matthews's similarity:  0.67
Gene Seymour's & Toby's similarity:  0.26
Michael Phillips's & Claudia Puig's similarity:  0.54
Michael Phillips's & Mick LaSalle's similarity:  0.39
Michael Phillips's & Jack Matthews's similarity:  0.32
Michael Phillips's & Toby's similarity:  0.39
Claudia Puig's & Mick LaSalle's similarity:  0.31
Claudia Puig's & Jack Matthews's similarity:  0.32
Claudia Puig's & Toby's similarity:  0.36
Mick LaSalle's & Jack Matthews's similarity:  0.29
Mick LaSalle's & Toby's similarity:  0.4
Jack Matthews's & Toby's simil

Почему бы не взять коэффициент корреляции?

<img src=https://wikimedia.org/api/rest_v1/media/math/render/svg/2b9c2079a3ffc1aacd36201ea0a3fb2460dc226f>

<img src=https://wikimedia.org/api/rest_v1/media/math/render/svg/b87fab4bd95646a6aa894efe96e894761c94498f>

где $n$ - размер выборки, <br>
$x_{i},y_{i}$ - оценки первого и второго пользователя <br>
$\bar {x}={\frac {1}{n}}\sum _{i=1}^{n}x_{i}$ выборочное среднее; аналогично для $\bar {y}$. 

In [8]:
def sim_pearson(reviews, person1, person2):
    stats = {}
    similar = list(set(reviews[person1].keys()) & set(reviews[person2].keys()))
    for person in [person1, person2]:
        stats.setdefault(person, {'ranks': 0, 'sum_ranks': 0, 'sum_ranks_sq': 0})
        stats[person]['ranks'] = [reviews[person][i] for i in similar]
        stats[person]['sum_ranks'] = np.sum(stats[person]['ranks'])
        stats[person]['sum_ranks_sq'] = np.sum([x ** 2 for x in stats[person]['ranks']])

    n = len(similar)
    sum_product = np.dot(stats[person1]['ranks'], stats[person2]['ranks'])
    numerator = n * sum_product - stats[person1]['sum_ranks'] * stats[person2]['sum_ranks']
    denominator = np.sqrt((n * stats[person1]['sum_ranks_sq'] - stats[person1]['sum_ranks'] ** 2) *
                          (n * stats[person2]['sum_ranks_sq'] - stats[person2]['sum_ranks'] ** 2))
    if denominator == 0:
        return 0
    r_coeff = numerator / denominator
    return r_coeff

In [9]:
compared = set()
for first in reviews:
    for second in reviews:
        compared.add((first, second))
        if first != second and (second, first) not in compared:
            print(f'{first} and {second} correlation similarity: ', round(sim_pearson(reviews, first, second), 2))

Lisa Rose and Gene Seymour correlation similarity:  0.4
Lisa Rose and Michael Phillips correlation similarity:  0.4
Lisa Rose and Claudia Puig correlation similarity:  0.57
Lisa Rose and Mick LaSalle correlation similarity:  0.59
Lisa Rose and Jack Matthews correlation similarity:  0.75
Lisa Rose and Toby correlation similarity:  0.99
Gene Seymour and Michael Phillips correlation similarity:  0.2
Gene Seymour and Claudia Puig correlation similarity:  0.31
Gene Seymour and Mick LaSalle correlation similarity:  0.41
Gene Seymour and Jack Matthews correlation similarity:  0.96
Gene Seymour and Toby correlation similarity:  0.38
Michael Phillips and Claudia Puig correlation similarity:  1.0
Michael Phillips and Mick LaSalle correlation similarity:  -0.26
Michael Phillips and Jack Matthews correlation similarity:  0.13
Michael Phillips and Toby correlation similarity:  -1.0
Claudia Puig and Mick LaSalle correlation similarity:  0.57
Claudia Puig and Jack Matthews correlation similarity:  0.

Давайте посмотрим на коэффициенты корреляции. У нас нашлось несколько примеров, у которых коэффициент корреляции равен 1. И есть даже сильная обратная линейная взаимосвязь: -1. 

Что можно делать с отрицательными корреляциями? Какие есть недостатки при использовании этой метрики?

In [95]:
reviews['Toby'], reviews['Michael Phillips']

({'Snakes on a Plane': 4.5,
  'You, Me and Dupree': 1.0,
  'Superman Returns': 4.0},
 {'Lady in the Water': 2.5,
  'Snakes on a Plane': 3.0,
  'Superman Returns': 3.5,
  'The Night Listener': 4.0})

Сортировка самих пользователей на основе схожести:

In [25]:
def sort_users(reviews, person, n=5, similarity=sim_pearson):
    scores = [(other, round(similarity(reviews, person, other),2)) for other in reviews if other != person]
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:n]

In [12]:
sort_users(reviews, 'Toby', n=5)

[('Lisa Rose', 0.99),
 ('Mick LaSalle', 0.92),
 ('Claudia Puig', 0.89),
 ('Jack Matthews', 0.66),
 ('Gene Seymour', 0.38)]

Перейдем к самим рекомендациям по найденным похожим пользователям.

Кстати, сколько выбирать соседей?

In [13]:
def get_recoms_by_users(prefs, person, similarity=sim_pearson):
    totals = {}
    sim_sums = {}
    for other in prefs:
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        if sim <= 0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item]*sim
                sim_sums.setdefault(item, 0)
                sim_sums[item] += sim

    rankings = [(item, round(total/sim_sums[item], 2)) for item, total in totals.items()]
    rankings = sorted(rankings, key=lambda x: x[1], reverse=True)
    
    return rankings

In [14]:
get_recoms_by_users(reviews, 'Toby', sim_distance)

[('The Night Listener', 3.46),
 ('Lady in the Water', 2.78),
 ('Just My Luck', 2.42)]

4. **Item-based similarity**. 

Попробуем получить рекомендации на основе похожести самих фильмов. 

Поменяем местами ключи с значениями в исходном словаре. Пример:

```
{user: {film:score}}
```
```
{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}
``` 
=>

```
{film: {user:score}}
```
```{'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}```

In [15]:
def transform_prefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            result[item][person] = prefs[person][item]
    return result

In [17]:
movies = transform_prefs(reviews)
movies

{'Lady in the Water': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.0,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0},
 'Snakes on a Plane': {'Lisa Rose': 3.5,
  'Gene Seymour': 3.5,
  'Michael Phillips': 3.0,
  'Claudia Puig': 3.5,
  'Mick LaSalle': 4.0,
  'Jack Matthews': 4.0,
  'Toby': 4.5},
 'Just My Luck': {'Lisa Rose': 3.0,
  'Gene Seymour': 1.5,
  'Claudia Puig': 3.0,
  'Mick LaSalle': 2.0},
 'Superman Returns': {'Lisa Rose': 3.5,
  'Gene Seymour': 5.0,
  'Michael Phillips': 3.5,
  'Claudia Puig': 4.0,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 5.0,
  'Toby': 4.0},
 'You, Me and Dupree': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.5,
  'Claudia Puig': 2.5,
  'Mick LaSalle': 2.0,
  'Jack Matthews': 3.5,
  'Toby': 1.0},
 'The Night Listener': {'Lisa Rose': 3.0,
  'Gene Seymour': 3.0,
  'Michael Phillips': 4.0,
  'Claudia Puig': 4.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0}}

In [20]:
get_recoms_by_users(movies, 'Just My Luck')

[('Michael Phillips', 4.0), ('Jack Matthews', 3.0)]

In [23]:
def get_similar_items(reviews, n=10):
    result = {}
    itemreviews = transform_prefs(reviews)
    counter = 0
    for item in tqdm(itemreviews):
        scores = sort_users(itemreviews, item, n=n, similarity=sim_distance)
        result[item] = scores
    return result

У нас всего 6 фильмов, посчитаем скоры похожести для всех пар:

In [33]:
itemsim = get_similar_items(reviews, n=5)
itemsim

100%|██████████| 6/6 [00:00<00:00, 5203.85it/s]


{'Lady in the Water': [('You, Me and Dupree', 0.45),
  ('The Night Listener', 0.39),
  ('Snakes on a Plane', 0.35),
  ('Just My Luck', 0.35),
  ('Superman Returns', 0.24)],
 'Snakes on a Plane': [('Lady in the Water', 0.35),
  ('The Night Listener', 0.32),
  ('Superman Returns', 0.31),
  ('Just My Luck', 0.26),
  ('You, Me and Dupree', 0.19)],
 'Just My Luck': [('Lady in the Water', 0.35),
  ('You, Me and Dupree', 0.32),
  ('The Night Listener', 0.3),
  ('Snakes on a Plane', 0.26),
  ('Superman Returns', 0.21)],
 'Superman Returns': [('Snakes on a Plane', 0.31),
  ('The Night Listener', 0.25),
  ('Lady in the Water', 0.24),
  ('Just My Luck', 0.21),
  ('You, Me and Dupree', 0.19)],
 'You, Me and Dupree': [('Lady in the Water', 0.45),
  ('Just My Luck', 0.32),
  ('The Night Listener', 0.29),
  ('Snakes on a Plane', 0.19),
  ('Superman Returns', 0.19)],
 'The Night Listener': [('Lady in the Water', 0.39),
  ('Snakes on a Plane', 0.32),
  ('Just My Luck', 0.3),
  ('You, Me and Dupree', 0.

In [31]:
def getRecommendedItems(reviews, item_match):
    scores = {}
    totalSim = {}
    const = 1e-7
    for item, rating in reviews.items():
        for similarity, item in item_match[item]:
            if item in reviews: 
                continue
            scores.setdefault(item, 0)
            scores[item] += similarity * rating
            totalSim.setdefault(item, 0)
            totalSim[item] += similarity
            if totalSim[item] == 0: 
                totalSim[item] = const # чтобы избежать деления на ноль

    rankings = [(score/totalSim[item], item) for item, score in scores.items()]
    rankings = sorted(rankings, key=lambda x: x[1], reverse=True)
    return rankings

In [36]:
getRecommendedItems(critics['Toby'], itemsim)

[(3.182634730538922, 'The Night Listener'), (2.5983318700614575, 'Just My Luck'), (2.4730878186968837, 'Lady in the Water')]


### Плюсы и минусы user/item similarity подходов:

Из недостатков:
1) Для расчетов нужны оценки пользователей <br>
2) Проблема холодного старта <br>
3) Ненадеждные корреляции <br>
4) Слабая обобщающая способность <br>

Из достоинств:
1) Простота подходов и имплементации<br>
2) Возможность дать понятную интерпретацию результатам <br> 
3) Хороши по качеству как бейзлайны <br>
4) Вычислительно незатратные <br>
5) Можно быстро делать обновления

## Рекомендация на данных MovieLens

Источник: http://grouplens.org/datasets/movielens/

In [37]:
def loadMovieLens(path='.'):

    movies = {}
    for line in open(path + '/u.item', errors='ignore'):
        idx, title = line.split('|')[:2]
        movies[idx] = title

    prefs = {}
    for line in open(path + '/u.data', errors='ignore'):
        user, movie_id, rating, ts = line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movie_id]] = float(rating)
    return prefs

In [ ]:
prefs = loadMovieLens()
prefs['87'] # user_id

In [39]:
itemsim = get_similar_items(prefs, n=50)

100%|██████████| 1664/1664 [00:40<00:00, 40.76it/s]


In [27]:
get_recommended_prefs(['87'], itemsim)[:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.888888888888889, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City (1998)'),
 (4.77777777

# Домашнее задание 1. Сравнение методов по сходству пользователей и по сходству объектов


1. Требуется реализовать вычисление ошибки **MAE** и **RMSE** на тестовых данных [MovieLens](http://grouplens.org/datasets/movielens/).  
В качестве данных обучения можно использовать файлы с расширением base, а тестирование качества провести на файле test: пары файлов u1.base и u1.test, ..., u5.base и u5.test. Каждая пара -- это разбиение данных на обучающие (80%) и тестовые (20%) данные для всех пользователей $u$.
2. Для каждого метода (user-based и item-based) постройте графики зависимости MAE и RMSE от числа соседей (диапазон от 1 до 100 с разумным шагом).
3. Сравните подходы на основе полученных результатов по аналогии с пунктами 1 и 2. 
4. Как изменяется величина MAE (RMSE) от числа выдаваемых рекомендаций (top-n): $n \in \{1,3,5,10,15,20,30,40,50,100\}$? 
5. Как Вы считаете, какие фильмы чаще рекомендуются - популярные с высокими оценками или редкие (те, которые редко оцениваются) с высокими оценками? Почему это происходит?
6. Что делать, если соседей (то есть, похожих на целевого пользователя или конкретный товар) мало? Нужно (и можно) ли как-то учитывать достоверность таких рекомендаций? 
7. Насколько различны списки из top-n рекомендаций? Попробуйте улучшить результаты подбором $\beta$ для минимизации MAE (RMSE) в гибридных рекомендациях в зависимости от числа соседей:
$$\beta\cdot r^{user-based}_{ui} + (1-\beta)\cdot r^{item-based}_{ui}, \mbox{ где } 0 \leq \beta \leq 1.$$ 

